In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows의 경우 '맑은 고딕'
# Mac의 경우: plt.rcParams['font.family'] = 'AppleGothic'
# Linux의 경우 적절한 한글 폰트를 설치해야 함 (예: 'NanumGothic')

# 음수 기호가 깨지는 문제 해결 (minus 폰트 설정)
plt.rcParams['axes.unicode_minus'] = False

In [3]:

# 엑셀 파일 불러오기
# 파일 경로를 지정하세요.
merged_df = pd.read_excel('merged_file.xlsx')  # A 데이터
df = pd.read_excel('df_b.xlsx')  # B 데이터

In [4]:
# '압연년월' 열을 datetime 형식으로 변환 (B 데이터에 필요)
df['압연년월'] = pd.to_datetime(df['압연년월']).dt.strftime('%Y-%m')  # '날짜' 열에서 년월을 문자열로 변환

# B 데이터 그룹화하여 불량율 계산
grouped = df.groupby(['외경', '압연년월']).agg({
    '투입중량': 'sum',
    '발생중량': 'sum'
}).reset_index()

# 불량율 계산
grouped['불량율'] = (grouped['발생중량'] / grouped['투입중량']) * 100

# 고유한 '외경' 값 가져오기
unique_conditions = grouped['외경'].unique()

# 그래프를 저장할 폴더 생성
output_folder = 'combined_graphs'  # 폴더 이름
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

# 각 필터 조건에 대해 그래프 생성
for condition in unique_conditions:
    filtered_a_df = merged_df[merged_df['지시외경'] == condition]  # A 그래프 데이터 필터링
    filtered_b_df = grouped[grouped['외경'] == condition]  # B 그래프 데이터 필터링

    if not filtered_a_df.empty and not filtered_b_df.empty:  # 두 데이터가 모두 있을 때만 그래프 그리기
        # '압연년월'을 문자열로 변환하여 정렬
        filtered_b_df = filtered_b_df.sort_values(by='압연년월')

        fig, ax1 = plt.subplots(figsize=(12, 6))

        # A 그래프 그리기 (왼쪽 y축)
        sns.boxplot(x='year_month', y='균열대.1', data=filtered_a_df, ax=ax1, color='lightgray')
        ax1.set_xlabel('압연년월')
        ax1.set_ylabel('Soaking_Temp', color='gray')
        ax1.tick_params(axis='y', labelcolor='gray')
        plt.xticks(rotation=90)
        plt.ylim(1240,1300)

        # A 그래프 중앙값 표시
        medians = filtered_a_df.groupby('year_month')['균열대.1'].median()
        for xtick, median in zip(ax1.get_xticks(), medians):
            ax1.text(xtick, median, f'{median:.0f}', 
                     ha='center', va='top', color='black', fontsize=6)


        # B 그래프 그리기 (오른쪽 y축)
        ax2 = ax1.twinx()  # 보조 y축 생성
        sns.lineplot(x='압연년월', y='불량율', data=filtered_b_df, ax=ax2, marker='o', color='darkred')
        ax2.set_ylabel('불량율 (%)', color='darkred')
        ax2.tick_params(axis='y', labelcolor='darkred')

        # 불량률 값 표시
        for index, row in filtered_b_df.iterrows():
            ax2.text(row['압연년월'], row['불량율'], f"{row['불량율']:.1f}", 
                     color='darkred', ha='center', va='bottom',fontsize=8)

        # 그래프 제목 및 레이아웃 설정
        plt.title(f'Combined Graph for 외경 {condition}')
        fig.tight_layout()  # 레이아웃 조정

        # 범례 추가
        ax1.legend(['soaking_temp(℃)'], loc='upper left', frameon=False)
        ax2.legend(['불량율(%)'], loc='upper right', frameon=False)

        # 그래프 저장
        plt.savefig(os.path.join(output_folder, f'combined_graph_{condition}.png'), dpi=300)
        plt.close()  # 그래프를 닫아 메모리 관리
                
# 완료 메시지
print("모든 그래프가 저장되었습니다.")

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
C:\Users\seah\AppData\Local\Temp\ipykernel_34120\4114741771.py:59: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()  # 레이아웃 조정
C:\Users\seah\AppData\Local\Temp\ipykernel_34120\4114741771.py:59: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()  # 레이아웃 조정
C:\Users\seah\AppData\Local\Temp\ipykernel_34120\4114741771.py:59: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()  # 레이아웃 조정
C:\Users\seah\AppData\Local\Temp\ipykernel_34120\4114741771.py:59: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations

모든 그래프가 저장되었습니다.
